# 첫번째 프로젝트

최대한 많은 리뷰를 수집하고 별점을 기준으로 3점 이상은 긍정 덧글, 3점 미만은 부정 덧글로 분류하여 데이터베이스에 저장하고자 한다.

자신이 관심을 갖는 상품 종류를 선정하여 네이버 쇼핑에서 5개 이상의 상품을 선정하여 상품의 이름을 데이터베이스 테이블에 저장하시오. 상품의 이름은 파이썬을 활용하지 않고 직접 INSERT문을 사용해도 좋습니다. 그리고 각 상품에 대한 모든 리뷰를 수집하시오. 수집된 리뷰를 데이터베이스 테이블에 저장하면서 별점 3점 이상은 리뷰는 긍정, 별점 3점 미만은 부정으로 표시하는 컬럼을 추가해야 합니다. 이 필드는 향후 분류분석(로지스틱회귀)에 활용될 수 있습니다.

요구사항
1. 상품 정보를 저장하기 위한 테이블과 리뷰를 저장하기 위한 테이블의 구조표와 테이블 생성 구문을 제시
하고 리뷰를 수집하기 위해 작성한 소스코드와 각 단계별 실행 결과 및 설명을 제시하시오. 
2. 프로그램의 수행 결과를 확인하기 위해 상품 이름과 리뷰 내용, 리뷰 작성일, 별점, 긍정/부정 분류 결
과 등의 데이터가 저장되어 있는 상태를 파이썬을 통해 조회하여 하나의 결과 집합으로 표시한 스크린샷
을 제시하세요.
3. 각각의 동작 단위를 2개 이상 모듈화 하세요.
4. 이 과정을 완성하여 발표자료를 작성후 제출하고 직접 발표를 수행합니다.
5. 과제를 수행하면서 겪은 에러 및 문제점을 모두 제시하고 이를 해결하기 위한 과정도 제시하세요. 
6. 이 과제를 수행하면서 느낀점을 서술하고 향후 자신이 어떤 부분을 보완해야 할지 제시하세요.

## #01. 준비과정

### [1] 패키지 참조

In [1]:
# ChromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# Chrome을 제어하기 위한 객체
from selenium import webdriver
# Chrome이 웹 페이지 로딩을 완료 할 때까지 최대 n초간 대기하는 기능.
from selenium.webdriver.support.ui import WebDriverWait
# HTML 파서
from bs4 import BeautifulSoup
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time
# HTML요소를 가져오는 방법을 지정하기 위한 구조체
from selenium.webdriver.common.by import By
# 파일 저장시 파일 이름 조정을 위한 모듈
import re
# 오라클연동을 위한 모듈
import cx_Oracle as cx
# 입력한 데이터의 결과를 보기 위한 모듈
from pandas import DataFrame


### [3] 네이버 쇼핑 페이지에서 상품명과 해당 상품의 URL 저장

#### (1)html 가져오기

In [22]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)

# 네이버 쇼핑 페이지로 이동
driver.get("https://shopping.naver.com/")
# 브라우저가 쇼핑 페이지로 이동하는 시간동안 대기
time.sleep(1) 

myhtml1 = driver.page_source
soup = BeautifulSoup(myhtml1)
content = soup.select('#content  div.shoppingHomeResponsive_category__ub5P_ > div > div.shoppingCategoryListResponsive_inner__qJbsp > ul:nth-child(1) > li:nth-child(5) > a')[0]['href']
driver.get(content)
time.sleep(3)
myhtml = driver.page_source
while myhtml == myhtml1:
    myhtml = driver.page_source
soup = BeautifulSoup(myhtml)
content = "https://shopping.naver.com" + soup.select('#content > div._2hV4tcE3Zy > div > ul > li:nth-child(2) > a')[0]['href']
driver.get(content)
time.sleep(3)

search = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR,"._1QmWvlfXgs"))
search.click()
searchWord = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR,"#searchWord"))
searchWord.clear() # 기존 입력값 삭제
searchWord.send_keys('밀키트')

searchButton = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR,"._9tjNwJRxqV"))
searchButton.click()
time.sleep(3)
myhtml = driver.page_source
soup = BeautifulSoup(myhtml)
content = soup.select(".product_wrap")[0].select("._3m7zfsGIZR")
while not content:
    time.sleep(0.1)
    myhtml = driver.page_source
    soup = BeautifulSoup(myhtml)
    content = soup.select(".product_wrap")[0].select("._3m7zfsGIZR")
# 원하는 소스를 가져오고 난 후 브라우저 종료
driver.close()




#### (2) 상품명, url 가져오기

In [79]:
# 상품명과 url을 수집하기 위해 빈 리스트 준비
Titles = []
product = []
for item in content[:5]: #5개만 가져오기
    # 같은 사이트 내에서는 도메인주소 없이 사용하므로 url에 추가할 도메인주소 준비
    domain = 'https://shopping.naver.com'
    # 상품명
    Title = item.select("._1J1f1i2vk0")[0].text
    # 해당 상품의 URL
    URL = domain + item.select("a")[1].attrs['href']
    Titles.append(Title)
    product.append({Title:URL})
print(Titles)
product


['피코크 우리집 돼지고기김치찌개 500g', '[피코크 X 초마] 짬뽕 밀키트 672g', '피코크 의정부식 부대찌개 700g', '[피코크] 냉이된장찌개 밀키트', '노브랜드 베트남쌀국수간편세트 202g']


[{'피코크 우리집 돼지고기김치찌개 500g': 'https://shopping.naver.com/window-products/emart/5899071138?tr=swsc'},
 {'[피코크 X 초마] 짬뽕 밀키트 672g': 'https://shopping.naver.com/window-products/emart/5908297039?tr=swsc'},
 {'피코크 의정부식 부대찌개 700g': 'https://shopping.naver.com/window-products/emart/5908735163?tr=swsc'},
 {'[피코크] 냉이된장찌개 밀키트': 'https://shopping.naver.com/window-products/emart/5908297926?tr=swsc'},
 {'노브랜드 베트남쌀국수간편세트 202g': 'https://shopping.naver.com/window-products/emart/5899538869?tr=swsc'}]

### [4] 데이터베이스 접속 및 SQL 실행 객체 생성

In [2]:
dbcon = cx.connect('hr',                 
                   'hr',                 
                   'localhost:1521/xe'   
)
cursor = dbcon.cursor()

### [5] 데이터베이스의 테이블 생성하기
각 상품당 리뷰번호를 시퀸스를 이용하여 저장하고 해당 번호를 기본키로 사용한다.

상품 테이블에 있는 이름을 외래키로 사용하여 테이블들을 연결한다.

![그림](sql_DDL.PNG)


In [ ]:
sql = '''
CREATE TABLE PRODUCT(
    PDNO NUMBER NOT NULL,
    PDNAME VARCHAR2(1200) NOT NULL,
    PRIMARY KEY(PDNO)
)
'''
cursor.execute(sql)
sql = 'CREATE SEQUENCE SEQ_PRODUCT START WITH 1 INCREMENT BY 1'

cursor.execute(sql)
sql = '''
CREATE TABLE REVIEW(
    RVNO NUMBER NOT NULL,
    RVCONTENT VARCHAR2(1200) NULL,
    RVDATETIME VARCHAR2(20) NOT NULL,
    RVVALUE NUMBER NOT NULL,
    RVRESULT VARCHAR2(10) NOT NULL,
    PDNO NUMBER NOT NULL,
    PRIMARY KEY (RVNO),
    FOREIGN KEY (PDNO) REFERENCES PRODUCT (PDNO)
)
'''
cursor.execute(sql)
sql = 'CREATE SEQUENCE SEQ_REVIEW START WITH 1 INCREMENT BY 1'
cursor.execute(sql)





## #02. 모듈화

사용할 함수를 정의하여 비동기처리를 통해 효율을 높인다.
- 상품명을 데이터베이스에 넣는 과정을 함수로 정의한다.
- 얻어온 리뷰를 데이터베이스로 넣는 과정을 함수로 정의한다.
- 리뷰를 얻어오는 과정을 함수로 정의한다.
- 조회?

### [1] 상품명을 데이터베이스로 넣는 함수 정의

In [55]:
def pdInsertDB(cursor, dbcon, title):
    sql = 'select pdname from product where pdname = :1'
    cursor.execute(sql, [title])
    if not cursor.fetchall(): # 이미 해당 데이터가 DB에 있으면 조회만하고 넘어간다.
        sql = f'INSERT INTO product (pdno, pdname) VALUES (seq_product.nextval, :1)'
        cursor.execute(sql, [title])
        print('입력되었습니다')
        dbcon.commit()

### [2] 얻어온 리뷰를 데이터베이스로 넣는 함수 정의

In [37]:
def rvInsertDB(cursor, dbcon, review_content, review_date, review_value, review_result, pdno):
    sql = f'select * from review where rvcontent = :1 and rvdatetime = :2 and rvvalue = :3'
    cursor.execute(sql,[review_content,review_date,review_value])
    result = cursor.fetchall()
    if not result: # 해당 리뷰가 DB에 없을때만 넣어준다.
        sql = f'INSERT INTO review (rvno, rvcontent, rvdatetime, rvvalue, rvresult, pdno) VALUES (seq_review.nextval, :1, :2, :3, :4, :5)'
        try:
            cursor.execute(sql, [review_content,review_date,review_value,review_result,pdno])
            dbcon.commit()
        except Exception as e:  
            print('오류가 발생했습니다',e)
            review_content = '' # 리뷰 내용이 DB에 들어가기 너무 큰 경우 내용 삭제
            try:
                cursor.execute(sql, [review_content,review_date,review_value,review_result,pdno])
                dbcon.commit()
            except Exception as e:
                print('오류가 발생했습니다',e)
        

### [3] 리뷰를 얻어올 함수 정의

In [38]:
def getReview(driver, dbcon, cursor, url, pdno):
    driver = webdriver.Chrome()
    # 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
    driver.implicitly_wait(5)
    driver.get(url)
    # 페이지 이동시간 대기
    time.sleep(1) 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    # 스크롤 이동시간 대기
    time.sleep(3)
    button = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR,"._2Q0vrZJNK1 a"))
    while not button: # 이동시간이 부족했다면 재실행
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") 
        time.sleep(3)
        button = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR,"._2Q0vrZJNK1 a"))
    button.click()
    time.sleep(3)
    myhtml = driver.page_source
    soup = BeautifulSoup(myhtml)
    total_review = int(re.sub('[^0-9]','',soup.select("._2Q0vrZJNK1 a")[0].text )) # 전체 리뷰수를 가져올때 `,` 또는 기타 다른 글자가 있을 시 삭제
    page_num = total_review//20 + 1 if total_review%20 else 0
    page = 0
    prev_review = '' # 이전페이지의 내용을 가져왔는지 아닌지 확인
    while page<page_num: # 리뷰가져오는건 페이지 넘버수만큼 다음버튼은 페이지 넘버수-1만큼 실행
        if page:
            next = WebDriverWait(driver, 3).until(lambda x:x.find_element(By.CSS_SELECTOR,"._1kSau7aQAr  ._2Ar8-aEUTq")) # 다음페이지 버튼
            next.click()
            time.sleep(0.3)
        myhtml1 = driver.page_source
        while prev_review == myhtml1: # 페이지로딩이 되기 전 이전소스를 가져오면 잠시 대기후 다시가져온다
            time.sleep(0.3)
            myhtml1 = driver.page_source
        prev_review = myhtml1
        soup1 = BeautifulSoup(myhtml1)
        REVIEW_ALL = soup1.select("._2DIJt-FWPQ")
        while not REVIEW_ALL:
            time.sleep(0.1)
            REVIEW_ALL = soup1.select("._2DIJt-FWPQ")
        REVIEW_ALL = REVIEW_ALL[0]
        review = REVIEW_ALL.select(".BnwL_cs1av")
        for rev in review:
            review_value = int(rev.select('em')[0].text.strip())
            review_date = rev.select("._2L3vDiadT9")[1].text.strip().replace('.','-')[:-1] # 마지막 `.` 제거
            review_content = rev.select('._1kMfD5ErZ6')[0].text.strip()
            review_result = '부정' if review_value < 3 else '긍정' # 별점에 따른 긍정/부정 결과
            time.sleep(0.001)
            rvInsertDB(cursor, dbcon, review_content, review_date, review_value, review_result, pdno) #DB입력
        page +=1
    driver.close()


## #03. 실행

### [1] 상품명을 데이터베이스에 입력

In [60]:
for title in Titles:
    pdInsertDB(cursor, dbcon, title)

### [2] 리뷰를 데이터베이스에 입력

In [62]:
for i,v in enumerate(product):
    title = Titles[i]
    url = v[title]
    # DB에서 상품코드 가져오기
    sql = 'select pdno from product where pdname = :1' 
    cursor.execute(sql,[title])
    pdno = cursor.fetchall()[0][0]
    getReview(driver, dbcon, cursor, url, pdno)

오류가 발생했습니다 ORA-12899: "HR"."REVIEW"."RVCONTENT" 열에 대한 값이 너무 큼(실제: 1587, 최대값: 1200)


### [3] 데이터베이스를 조회하여 하나의 결과 집합으로 표현

In [51]:
sql = '''
SELECT
  p.pdname as 상품명,
  ' ',
  ' ',
  ' ',
  ' ',
  COUNT(CASE WHEN r.rvresult = '긍정' THEN r.pdno END) AS 긍정_개수,
  COUNT(CASE WHEN r.rvresult = '부정' THEN r.pdno END) AS 부정_개수
FROM
  review r
JOIN
  product p ON r.pdno = p.pdno
GROUP BY
  p.pdname
'''
cursor.execute(sql)
result = cursor.fetchall()
sql = '''
SELECT p.pdname as 상품명, r.rvcontent as 리뷰내용, r.rvdatetime as 리뷰날짜, r.rvvalue as 별점, r.rvresult as 분류_결과,' ',' '
FROM review r
JOIN product p
ON p.pdno = r.pdno
'''
cursor.execute(sql)
result += cursor.fetchall()

df = DataFrame(result, columns=['상품명', '리뷰내용', '리뷰날짜','별점','긍정/부정 분류 결과','긍정_개수','부정_개수'])
df


,상품명,리뷰내용,리뷰날짜,별점,긍정/부정 분류 결과,긍정_개수,부정_개수
0,[피코크 X 초마] 짬뽕 밀키트 672g,,,,,1313,23
1,피코크 의정부식 부대찌개 700g,,,,,3278,27
2,피코크 우리집 돼지고기김치찌개 500g,,,,,737,4
3,노브랜드 베트남쌀국수간편세트 202g,,,,,3048,9
4,[피코크] 냉이된장찌개 밀키트,,,,,217,1
...,...,...,...,...,...,...,...
8657,피코크 의정부식 부대찌개 700g,양은 2인분 정도 괜찮아요...,22-02-21,5,긍정,,
8658,피코크 의정부식 부대찌개 700g,배송도 빠르고 양도 넉넉해요,22-02-18,5,긍정,,
8659,피코크 의정부식 부대찌개 700g,다른 부대찌개도 많지만~ 울집은 의정부부대찌개를 젤 맛있어해요 ㅎㅎㅎ,22-01-27,5,긍정,,
8660,피코크 의정부식 부대찌개 700g,재구매내용물도 많고 맛있어요,22-02-21,5,긍정,,


## #04. 과제 수행시 겪은 시행착오 및 보완사항
### [1] 시행착오
#### (1) 비동기처리

작업의 효율성을 위해 리뷰를 DB에 넣는 작업을 비동기처리로 작업하려 하였으나 DB에 일부 저장되지 않는 경우가 계속해서 발생하였음

비동기처리를 하지 않았음에도 작업속도 차이가 유의미한 차이를 보이지 않아서 그냥 뺌





### [2] 보완점
#### (1)